<a href="https://colab.research.google.com/github/kowalski-k/projekt_TOM/blob/master/Projekt_TOM_U_NET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importowanie używanych bibliotek

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os

#sprawdzenie używanej wersji tensorflow
print(tf.__version__)

# Pobranie danych i zapoznanie się z nimi

In [0]:
#pobieranie danych
! curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
! sudo apt-get install git-lfs
! git lfs install
! git clone https://github.com/neheller/kits19.git 
%cd kits19
! python -m starter_code.get_imaging
!git checkout feature/google-colab
!make

In [0]:
#wczytanie danych dla jednego wybranego pacjenta
from starter_code.utils import load_case
volume, segmentation = load_case("case_00113")

In [0]:
#wizualizacja nowotworów i nerek na tle obrazu CT (zobaczyć jedynie jak działa, przyda się na końcu)
'''
visualize("case_00113", "case_113")
''' 

In [0]:
#wyodrębnienie i obrazowanie segementation
segmentation = segmentation.get_fdata()
(a,x,y) = segmentation.shape
for i in range(0,a,1):
  plt.figure()
  plt.imshow(segmentation[i],'gray')
  plt.title(i)

In [0]:
#wyodrębnienie i wczytywanie "czystych" obrazow bez zaznaczen (volume)
volume = volume.get_fdata()
(a,x,y) = volume.shape
for i in range(0,a,1):
  plt.figure()
  plt.imshow(volume[i],'gray')
  plt.title(i)

In [0]:
#zdobycie przydatnych w późniejszej części informacji o segmentation
i = 19
print(segmentation.shape)
print(type(segmentation[i]))
print(np.amax(segmentation[i]))
print(np.amin(segmentation[i]))
print(segmentation[i])
plt.imshow(segmentation[i],'gray')

In [0]:
#zdobycie przydatnych w późniejszej części informacji o volume
print(volume.shape)
print(type(volume[0]))
print(np.amax(volume[0]))
print(np.amin(volume[0]))
print(volume[0])
plt.imshow(volume[0],'gray')